In [ ]:
!pip install gradio faster-whisper speechrecognition jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 60.9 MB/s eta 0:00:00


In [ ]:
import speech_recognition as sr
from faster_whisper import WhisperModel
from pydub import AudioSegment
import jiwer

In [ ]:
model_path = "large-v3"

model = WhisperModel(model_path, device="cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

In [ ]:
recognizer = sr.Recognizer()

In [ ]:
def recognize_speech_whisper(audio_file_path):
    try:
        segments, info = model.transcribe(audio_file_path, beam_size=5)
        transcription = "".join([segment.text for segment in segments])
        return transcription
    except Exception as e:
        return None

In [ ]:
def recognize_speech_sr(audio_file_path):
    audio = AudioSegment.from_mp3(audio_file_path)
    temp_file = "temp.wav"
    audio.export(temp_file, format="wav")

    with sr.AudioFile(temp_file) as source:
        audio_data = recognizer.record(source)
        transcription = recognizer.recognize_google(audio_data)
        return transcription

In [ ]:
audio_path = "/content/Things fall apart.mp3"

In [ ]:
ground_truth = """
                Chapter One


Okonkwo was well known throughout the nine villages and even beyond. His fame rested on solid personal achievements. As a young man of eighteen he had brought honor to his village by throwing Amalinze the Cat. Amalinze was the great wrestler who for seven years was unbeaten, from Umuofia to Mbaino. He was called the Cat because his back would never touch the earth. It was this man that Okonkwo threw in a fight which the old men agreed was one of the fiercest since the founder of their town engaged a spirit of the wild for seven days and seven nights.
              """

In [ ]:
whisper_transcription = recognize_speech_whisper(audio_path)
print(whisper_transcription)

 Chapter 1 Okonkwo was well known throughout the nine villages and even beyond. His fame rested on solid personal achievements. As a young man of 18 he had brought honor to his village by throwing Amalins the cat. Amalins was the great wrestler who for seven years was unbeaten from Umofia to Mbeno. He was called the cat because his back would never touch the earth. It was this man that Okonkwo threw in a fight which the old men agreed was one of the fiercest since the founder of their town engaged a spirit of the wild for seven days and seven nights.


In [ ]:
sr_transcription = recognize_speech_sr(audio_path)
print(sr_transcription)

Chapter One Ok Google was well known throughout the nine Villages and even Beyond his famous restaurant on solid personal achievements as a young man of 18 he had brought honor to his village by throwing Emily's the cat Ameren was the great wrestler who for 7 years was unbeaten from Yuma to Beno he was called the cat because his back would never touch the Earth it was this man that Okonkwo in a fight which the old man agreed was one of the first since the founder of the Town engaged the spirit of the wild for seven days and seven nights


In [ ]:
def calculate_WER(ground_truth, transcription):
    wer = jiwer.wer(ground_truth, transcription)
    return wer

In [ ]:
whisper_WER = calculate_WER(ground_truth, whisper_transcription)
sr_WER = calculate_WER(ground_truth, sr_transcription)

In [ ]:
print(f"Whisper WER: {whisper_WER*100:.2f}%")
print(f"Google Speech Recognition WER: {sr_WER*100:.2f}%")

Whisper WER: 8.65%
Google Speech Recognition WER: 26.92%
